In [118]:
input_dir = '/Users/augustineigboke/Documents/Image-Segmentation/6dicomscans'
output_dir = '/Users/augustineigboke/Documents/Image-Segmentation/LiTs24/dicom_groups'
nifti_output_dir = '/Users/augustineigboke/Documents/Image-Segmentation/LiTs24/dicom_groups'

In [149]:
from glob import glob
import shutil
import os
import sys
import dicom2nifti

In [156]:
class ImageSementationUtils:
    def __init__(self, input_dir, output_dir):
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.volumes = glob(self.input_dir + '/*')
        self.group_size = self.get_group_size()

    def get_group_size(self):
        min = sys.maxsize
        for volume in glob(self.input_dir + '/*'):
            l = len(glob(volume + '/*'))
            min = l if l != 0 and l < min else min
        return min

    def prepare_dicom(self):
        group_folders = self.create_output_dirs(self.group_size)
        if len(group_folders) > 0:
            self.move_volumes_to_output_folders(group_folders)
        else:
            print('Folder(s) are empty')
        

    def create_output_dirs(self, group_size):
        group_folders = []
        if group_size <= 0:
            return group_folders
        
        for vol in self.volumes:
            vol_name = os.path.basename(os.path.normpath(vol))
            num_groups = int(len(glob(vol + '/*')) / group_size)
            for i in range(num_groups):
                group_folder = os.path.join(self.output_dir, vol_name + '_' + str(i))
                if os.path.exists(group_folder):
                    shutil.rmtree(group_folder)
                os.mkdir(group_folder)
                group_folders.append(group_folder)
        return group_folders

    def move_volumes_to_output_folders(self, group_folders):
        current_folder = 0
        for vol in self.volumes:
            num_folder_groups = int(len(glob(vol + '/*')) / self.group_size)
            files_in_folder = sorted(glob(vol + '/*'))
            
            start = 0
            for folder in range(num_folder_groups):
                self.move_files_to_output_dir(files_in_folder[start : start + self.group_size], group_folders[current_folder])
                start = start + self.group_size
                current_folder = current_folder + 1
    
    def move_files_to_output_dir(self, files, group_folder):
        group_size = self.get_group_size()
        for i, file in enumerate(files):
            if i > self.group_size:
                break
            shutil.move(file, group_folder)

    def dicom2nifti(self, nitfi_ouput_dir):
        reader = sitk.ImageSeriesReader()
        for dicom_group in glob(self.output_dir + '/*'):
            dicom_group_name = os.path.basename(os.path.normpath(dicom_group))
            dicom2nifti.dicom_series_to_nifti(dicom_group, os.path.join(nitfi_ouput_dir, dicom_group_name + '.nii.gz'))
        

In [151]:
img_seg_utils = ImageSementationUtils(input_dir, output_dir)
group_size = img_seg_utils.get_group_size()
print(group_size)
img_seg_utils.prepare_dicom()


75


In [152]:
img_seg_utils.dicom2nifti(nifti_output_dir)